In [6]:
from bs4 import BeautifulSoup
import requests
import json
import urllib

def page_item_list(page, adult, child, checkin, checkout, keyword):
    base_url = "https://www.yanolja.com/api/v1/contents/search"
    params = {
        'capacityAdult': adult,
        'capacityChild': child,
        'page': page,
        'limit': 20,
        'keyword': keyword,
        'checkinDate': checkin,
        'checkoutDate': checkout,
        'searchKeyword': keyword,
        'advert': 'KEYWORD',
        'searchType': 'all',
        'all': 1,
        'sort': 100,
        'themes': None,
        'lat': '37.50681',
        'lng': '127.06624',
        'capacityChildAges': None  
    }

    headers = {
        'Referer' : 'https://www.yanolja.com/search/{0}?placeListType=search&keyword={0}'.format(urllib.parse.quote(keyword)),
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    #     'Cookie': 'ACEUCI=1; %2Fsearch%2F%EC%A0%9C%EC%A3%BC%EB%8F%84%2Fkeyword-all={%22keyword%22:%22%EC%A0%9C%EC%A3%BC%EB%8F%84%22%2C%22searchKeyword%22:%22%EC%A0%9C%EC%A3%BC%EB%8F%84%22%2C%22checkinDate%22:%222020-08-16%22%2C%22checkoutDate%22:%222020-08-17%22%2C%22capacityAdult%22:1%2C%22capacityChild%22:0%2C%22capacityChildAges%22:%22%22}; cgntId=ap-northeast-2%3A2b47caae-3694-44f0-9d2b-43c98b30c098; yanolja_sid=s%3A0K-pijLPMujuW6NI5cJVcGOYBO2FXY0h.ZwiC623ThkJsrpFM9m7MiLj%2F8qYFHr3272Xla0K8NCk; SavedFiltersKey=/search/%EC%A0%9C%EC%A3%BC%EB%8F%84/keyword-all; ACEUACS=1596521299174217533; ACEFCID=UID-5F28FB542D0B4D05779333B5; _fbp=fb.1.1596521300441.1991528559; _ga=GA1.2.1408952294.1596521300; _gid=GA1.2.788840749.1596521300; location={%22latitude%22:%2237.50681%22%2C%22longitude%22:%22127.06624%22%2C%22address%22:%22%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EA%B0%95%EB%82%A8%EA%B5%AC%20%ED%85%8C%ED%97%A4%EB%9E%80%EB%A1%9C108%EA%B8%B8%2042%22%2C%22addressShort%22:%22%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EA%B0%95%EB%82%A8%EA%B5%AC%20%ED%85%8C%ED%97%A4%EB%9E%80%EB%A1%9C108%EA%B8%B8%22%2C%22addressOnlyRoad%22:%22%ED%85%8C%ED%97%A4%EB%9E%80%EB%A1%9C108%EA%B8%B8%22}; wcs_bt=ae93a192ec48a4:1596521393; AWSALB=zm/Jfq0CVy83/Q/bcMcY2ADivgyVdh6KNQc6bG1tPs9SA7T5733p2wvOClRpxNMSodWkruizJEGIcJbThOG1sBYm+WURo+8cqojOGppWN4ykkwQMNbQuiwexg5Nx; AWSALBCORS=zm/Jfq0CVy83/Q/bcMcY2ADivgyVdh6KNQc6bG1tPs9SA7T5733p2wvOClRpxNMSodWkruizJEGIcJbThOG1sBYm+WURo+8cqojOGppWN4ykkwQMNbQuiwexg5Nx'
    }

    ## 호텔리스트가 홈페이지랑 다르게 나옴 -> 쿠키가 계속 갱신되는 것을 방지하기 위해 -> 처음 홈페이지에 들어가자마자의 쿠키를 저장해놓는다.
    s = requests.Session()
    s.get("https://www.yanolja.com/search/{0}?placeListType=search&keyword={0}".format(keyword))
    resp = s.get(base_url, params=params, headers=headers)
    # resp = requests.get(base_url, params = params, headers=headers)
    
    
    item_li = json.loads(resp.text)
    
    item_list = []
    #name_list = []
    #score_list = []
    #price_list = []


    for i in range(20):

        # 호텔 이름

        name = item_li['motels']['lists'][i]['name']
        #name_list.append(name)
    #print(name_list)



        # 호텔 점수

        score = item_li['motels']['lists'][i]['reviewScore']
        #score_list.append(score)
    #print(score_list)



        # 리뷰 개수

        count = item_li['motels']['lists'][i]['reviewCounts']
        #count_list.append(count)
    #print(count_list)



        # 호텔 가격 = > 예약마감이면 예약마감이 뜬다.

        if len(item_li['motels']['lists'][i]['displayPrices']) == 1:
            if item_li['motels']['lists'][i]['displayPrices'][0]['priceNote'] == '예약마감':
                price = '예약마감'
                #price_list.append(price)

            else:
                price = item_li['motels']['lists'][i]['displayPrices'][0]['rawDiscountPrice']
                #price_list.append(price)

        elif len(item_li['motels']['lists'][i]['displayPrices']) >= 2:
            for j in range(2):
                if item_li['motels']['lists'][i]['displayPrices'][j]['badgeText'] == '숙박특가':
                    if item_li['motels']['lists'][i]['displayPrices'][j]['priceNote'] == '예약마감':
                        price = '예약마감'
                        #price_list.append(price)

                    else:
                        price = item_li['motels']['lists'][i]['displayPrices'][j]['rawDiscountPrice']
                        #price_list.append(price)
    #print(price_list)



        item_list.append({'name' : name, 
                          'score' : score,
                          'count' : count,
                          'price' : price})
    print(item_list)    
    

    
    
page_item_list(1, 1, 0, '2020-08-16', '2020-08-17', '제주도')    

[{'name': '제주 해피니스펜션 (오션뷰,18.3월신축)', 'score': '4.6', 'count': '149', 'price': 59000}, {'name': '호텔 시리우스 제주', 'score': '4.6', 'count': '682', 'price': '예약마감'}, {'name': '서귀포 수펜션', 'score': '4.7', 'count': '53', 'price': 59800}, {'name': '유탑유블레스호텔 제주함덕', 'score': '4.6', 'count': '471', 'price': 89000}, {'name': 'HY 초이호스텔 제주', 'score': '4.4', 'count': '75', 'price': 40000}, {'name': '호텔 리젠트마린', 'score': '4.7', 'count': '926', 'price': 112000}, {'name': '제주 빈티지1950카페펜션(조식제공)', 'score': '4.8', 'count': '290', 'price': '예약마감'}, {'name': '제주 컬리넌 호텔', 'score': '4.4', 'count': '1213', 'price': 49500}, {'name': '서귀포 라운지하우스제주다펜션', 'score': '5', 'count': '2', 'price': 65000}, {'name': '서귀포 펜션연리', 'score': '5', 'count': '37', 'price': '예약마감'}, {'name': '아트스테이 서귀포 하버', 'score': '4.6', 'count': '2070', 'price': 51000}, {'name': '빠레브 호텔 제주', 'score': '4.5', 'count': '293', 'price': 51030}, {'name': '라마다 프라자 제주', 'score': '4.7', 'count': '179', 'price': 189992}, {'name': '봄그리고가을 호텔&리조트', 'score': '4.6'

#### 함수 변수 : 페이지 수, 성인 수, 아동 수, 체크인 날짜, 체크아웃 날짜, 키워드 

#### 한 페이지에 20개!!

In [5]:
page_item_list(1, 1, 0, '2020-08-16', '2020-08-17', '제주도')

[{'name': '제주 해피니스펜션 (오션뷰,18.3월신축)', 'score': '4.6', 'count': '149', 'price': 59000}, {'name': '호텔 시리우스 제주', 'score': '4.6', 'count': '682', 'price': '예약마감'}, {'name': '서귀포 수펜션', 'score': '4.7', 'count': '53', 'price': 59800}, {'name': '유탑유블레스호텔 제주함덕', 'score': '4.6', 'count': '471', 'price': 89000}, {'name': 'HY 초이호스텔 제주', 'score': '4.4', 'count': '75', 'price': 40000}, {'name': '호텔 리젠트마린', 'score': '4.7', 'count': '926', 'price': 112000}, {'name': '제주 빈티지1950카페펜션(조식제공)', 'score': '4.8', 'count': '290', 'price': '예약마감'}, {'name': '제주 컬리넌 호텔', 'score': '4.4', 'count': '1213', 'price': 49500}, {'name': '서귀포 라운지하우스제주다펜션', 'score': '5', 'count': '2', 'price': 65000}, {'name': '서귀포 펜션연리', 'score': '5', 'count': '37', 'price': '예약마감'}, {'name': '아트스테이 서귀포 하버', 'score': '4.6', 'count': '2070', 'price': 51000}, {'name': '빠레브 호텔 제주', 'score': '4.5', 'count': '293', 'price': 51030}, {'name': '라마다 프라자 제주', 'score': '4.7', 'count': '179', 'price': 189992}, {'name': '봄그리고가을 호텔&리조트', 'score': '4.6'